In [41]:
import pandas as pd
import numpy as np
import random
import tensorflow as tf

SEED=12
random.seed(SEED)
np.random.seed(SEED)
tf.random.set_seed(SEED)
print("시드고정:", SEED)

시드고정: 12


In [42]:
from google.colab import drive
drive.mount('/gdrive')

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).


데이터 전처리

In [43]:
drive_path = "/gdrive/My Drive/"

train = pd.read_csv(drive_path + "wine/train.csv")
test = pd.read_csv(drive_path + "wine/test.csv")
submission = pd.read_csv(drive_path + "wine/sample_submission.csv")

print(train.shape, test.shape, submission.shape)

(5497, 14) (1000, 13) (1000, 2)


In [44]:
train.head(2)

#목표 변수는 와인 품질을 나타내는 quality 이다.

,index,quality,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,type
0,0,5,5.6,0.695,0.06,6.8,0.042,9.0,84.0,0.99432,3.44,0.44,10.2,white
1,1,5,8.8,0.610,0.14,2.4,0.067,10.0,42.0,0.99690,3.19,0.59,9.5,red


train['type']

In [45]:
train['type'].value_counts()

#type열의 범주형 데이터는 문자열 값을 갖는다.
#모델 학습에 입력하기 위해선 숫자형 데이터로 변환해야함


white    4159
red      1338
Name: type, dtype: int64

In [46]:
#white를 1로 red를 0으로 바꾸자
train['type'] = np.where(train['type']=='white',1,0).astype(int)  #조건이 맞는건 1로 아닌건 0으로!
test['type'] = np.where(test['type']=='white',1,0 ).astype(int)  #여기는 test 위는 train
train['type'].value_counts()

1    4159
0    1338
Name: type, dtype: int64

train['quality']

In [47]:
train['quality'].value_counts()

#6등급 와인의 개수가 가장 많네

6    2416
5    1788
7     924
4     186
8     152
3      26
9       5
Name: quality, dtype: int64

목표 변수는 연속형 숫자 데이터가 아니라, 와인 등급을 나타내는 범주형 데이터이다.
따라서 케라스 to_categorical함수를 이용하여 목표 변수를 원핫 인코딩 변환하자

원-핫 인코딩은 단어 집합의 크기를 벡터의 차원으로 하고, 표현하고 싶은 단어의 인덱스에 1의 값을 부여하고, 다른 인덱스에는 0을 부여하는 단어의 벡터 표현 방식

와인 등급이 3~9까지 모두 7개 클래스로 구분되는데

0~6범위로 바꾸자 (3을 뺀 값)

:바꾼 이유는 바로 3~9 범위 값으로 원핫 인코딩 하면 숫자 0부터 최대값인 9까지 구분하기 때문

In [48]:
from tensorflow.keras.utils import to_categorical

y_train = to_categorical(train.loc[:,'quality'] -3)
y_train

array([[0., 0., 1., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [49]:
#1. 모델 학습에 사용할 피처 선택
#2. MinMax 스케일링으로 모든 피처 변수의 데이터를 0~1 범위로 정규화 변환!
#3. 훈련 데이터(X_train)로 정규화 학습을 하고, 같은 조건을 검증 데이터(X_test)에 적용하여 변환하자

#1
X_train = train.loc[:,"fixed acidity":]
X_test = test.loc[:,"fixed acidity":]

#2 피처 스케일링
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scaler.fit(X_train)
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.fit_transform(X_test)

print(X_train_scaled.shape, y_train.shape)
print(X_test_scaled.shape)

(5497, 12) (5497, 7)
(1000, 12)


모델 설계: 드랍아웃 활용

완전 연결 레이어(Dense) 4개 층으로 구성되는 신경망 모델을 구성하자.

과대적합을 방지하기 위하여 드랍아웃 레이어를 추가한다.

드랍아웃이란

:입력 레이어와 은닉 레이어 간의 연결 중 일부를 랜덤으로 제거한 상태에서 학습하는 기법!

:유닛 사이에 연결된 가중치 수를 줄이는 효과를 얻기 때문에 과대적합을 방지할 수 있다.

미니 배치 단위로 학습할 때마다 연결 네트워크에서 제거되는 가중치가 달라지므로 매번 네트워크 구조를 갖는 모델을 얻게 된다.

즉 앙상블 효과가 있어 모델 성능이 개선된다.

In [50]:
#심층 신경망 모델
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Dropout

def build_model(train_data, train_target):
  model = Sequential()
  model.add(Dense(128, activation='tanh', input_dim=train_data.shape[1] ))  #input_dim = 12
  model.add(Dropout(0.2))   #20퍼 확률로 랜덤하게 연결을 제거
  model.add(Dense(64, activation='tanh'))
  model.add(Dropout(0.2))
  model.add(Dense(32,activation='tanh'))
  model.add(Dense(train_target.shape[1], activation='softmax')) #출력 레이어의 활성화 함수 softmax

  model.compile(optimizer='RMSProp', loss='categorical_crossentropy',metrics=['acc','mae']) #metrics 옵션: 여러 개의 보조 평가지표(정확도와 평균절대값오차)

  return model

model = build_model(X_train_scaled, y_train)
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_11 (Dense)            (None, 128)               1664      
                                                                 
 dropout_6 (Dropout)         (None, 128)               0         
                                                                 
 dense_12 (Dense)            (None, 64)                8256      
                                                                 
 dropout_7 (Dropout)         (None, 64)                0         
                                                                 
 dense_13 (Dense)            (None, 32)                2080      
                                                                 
 dense_14 (Dense)            (None, 7)                 231       
                                                                 
Total params: 12231 (47.78 KB)
Trainable params: 12231

tanh함수는 -1 ~ 1 사이의 출력 범위를 갖는다.
입력값이 0 근처일 때는 학습률이 좋지만 입력값이 커지거나 작아지는 경우 기울기가 0에 가까워지므로 학습이 이루어지지 않는 문제가 생긴다.

ReLU함수에 비하여 사용 빈도가 낮은편

콜백 함수: Early Stopping 기법

모델 학습 과정을 세밀하게 컨트롤 할 수 있다.

딥러닝 모델 학습에서 에포크 수를 늘려 학습을 계속 반복하면 훈련 데이터에 대한 오차를 계속 낮출 수 있다. 하지만 과대적합을 일으켜 테스트 데이터를 포함한 새로운 데이터에 대한 예측력이 나빠지는 문제가 발생한다.

Early Stopping을 사용하면 과대적합이 발생하기 직전에 학습을 멈출 수 있다.

홀드 아웃으로 검증 데이터를 분할하고 검증 데이터에 대한 모델 성능이 일정 에포크 동안 좋아지지 않으면 모델 학습을 중단한다.

이때 허용되는 에포크 수를 patience옵션에 설정한다.

In [51]:
#200에포트로 설정되어 있지만, 학습 중 10에포크 동안 연속하여 검증 데이터에 대한 손실함수가 줄어들지 않으면 학습을 멈추도록 하자
#Early Stopping
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import EarlyStopping

X_tr, X_val, y_tr, y_val = train_test_split(X_train_scaled, y_train, test_size = 0.15,
                                            shuffle = True, random_state=SEED)
early_stopping = EarlyStopping(monitor='val_loss', patience=10)
history = model.fit(X_tr,y_tr, batch_size=64, epochs=200,
                    validation_data= (X_val,y_val),
                    callbacks=[early_stopping],
                    verbose=2)


Epoch 1/200
73/73 - 2s - loss: 1.2949 - acc: 0.4606 - mae: 0.1935 - val_loss: 1.1580 - val_acc: 0.5333 - val_mae: 0.1806 - 2s/epoch - 23ms/step
Epoch 2/200
73/73 - 0s - loss: 1.1731 - acc: 0.5043 - mae: 0.1780 - val_loss: 1.1043 - val_acc: 0.5442 - val_mae: 0.1731 - 365ms/epoch - 5ms/step
Epoch 3/200
73/73 - 0s - loss: 1.1421 - acc: 0.5113 - mae: 0.1739 - val_loss: 1.0815 - val_acc: 0.5394 - val_mae: 0.1682 - 364ms/epoch - 5ms/step
Epoch 4/200
73/73 - 0s - loss: 1.1250 - acc: 0.5169 - mae: 0.1719 - val_loss: 1.1197 - val_acc: 0.5030 - val_mae: 0.1704 - 361ms/epoch - 5ms/step
Epoch 5/200
73/73 - 0s - loss: 1.1182 - acc: 0.5205 - mae: 0.1712 - val_loss: 1.0707 - val_acc: 0.5552 - val_mae: 0.1682 - 354ms/epoch - 5ms/step
Epoch 6/200
73/73 - 0s - loss: 1.1082 - acc: 0.5233 - mae: 0.1712 - val_loss: 1.0619 - val_acc: 0.5527 - val_mae: 0.1670 - 234ms/epoch - 3ms/step
Epoch 7/200
73/73 - 0s - loss: 1.1020 - acc: 0.5298 - mae: 0.1704 - val_loss: 1.0593 - val_acc: 0.5503 - val_mae: 0.1661 - 190

early stopping 으로 학습이 중지가 되면 학습이 중지된 상태의 가중치로 고정됨

In [52]:
model.evaluate(X_val,y_val)

26/26 [==============================] - 0s 3ms/step - loss: 1.0264 - acc: 0.5600 - mae: 0.1625


[1.0263590812683105, 0.5600000023841858, 0.16253085434436798]

예측값 정리 및 파일 제출

In [53]:
#test 데이터에 대한 예측값 정리
y_pred_proba = model.predict(X_test)
y_pred_proba[:5]

32/32 [==============================] - 0s 2ms/step


array([[0.35855773, 0.01398872, 0.3020564 , 0.22041343, 0.02329257,
        0.07057843, 0.0111127 ],
       [0.48345685, 0.00493023, 0.19034454, 0.1967042 , 0.04671643,
        0.06400549, 0.0138423 ],
       [0.2928567 , 0.01337365, 0.37213737, 0.23877831, 0.02097888,
        0.05587856, 0.00599657],
       [0.20346418, 0.02641708, 0.4308104 , 0.252443  , 0.03312429,
        0.04674061, 0.00700038],
       [0.39057308, 0.00903073, 0.251754  , 0.24526152, 0.03356467,
        0.06036998, 0.00944606]], dtype=float32)

In [54]:
#처음에 3을 뺐으니 다시 더해야지
y_pred_label = np.argmax(y_pred_proba,axis=-1) +3
y_pred_label[:5]

array([3, 3, 5, 5, 3])

제출

In [55]:
submission['quality'] = y_pred_label.astype(int)
submission.head()

,index,quality
0,0,3
1,1,3
2,2,5
3,3,5
4,4,3


In [56]:
submission.to_csv(drive_path+"wine/wine_dnn_001.csv",index=False)